# Building an Ocean model

The objective: understand the basics behind a simple particle tracking simulation using Oceananigans.jl

## What is Oceananigans.jl?

***Oceananigans.jl*** is an ocean modeling library built in the Julia programming language. <br>
Oceananigans is designed to be user-friendly and flexible and is capable of performing all types of <br>
simulations, from _small-scale_ "isotropic" simulations to _large-scale_ realistic oceans! <br>

In this lab, you will learn the syntax of Oceananigans.jl and bits of ***Julia*** (data structures, load data, plotting) <br>
mainly in the context of the Oceananigans package. You will learn how to set <br>
up and run a simple simulation with Oceananigans.

We will go through all the Oceananigans' building blocks (if time allows it!):
1) Grids (this module)<br>
2) Fields (this module)<br>
3) Models (module 2)<br>
4) Simulation (module 2)<br>

If you are interested in learning more about Julia and Oceananigans or contributing get in touch wither through <br>
email or on the Oceananigans web page!

# Install a package, load and visualize data

## Packages in julia

Like Python, also Julia relies on a package system. <br>

Julia provides an in-build package manager to simplify adding and loading packages. <br>
For example, to install the package "Downloads" (https://github.com/JuliaLang/Downloads.jl) it is sufficient to run `Pkg.add("Downloads")`. <br>
Once installed, we can use the package through `using Downloads`

In [ ]:
using Pkg
Pkg.add("Downloads")
using Downloads

Oceananigans is already installed on the hub, so it is not 
necessary to `Pkg.add("Oceananigans")`. 

We can use Oceananigans with

In [ ]:
# Pkg.add("Oceananigans")
using Oceananigans

### First step: load a bathymetry file

The ocean exists in a complex domain: it has a variable seabed height and is bounded by continents and islands. <br> 
All this information is visualized through a _bathymetry_ map. <br>
The term "bathymetry" refers to the ocean's depth relative to sea level. <br>
In the same way, a _topographic_ map describes the elevation of the terrain, a _bathymetric_ map illustrates the land that lies underwater. 

The bathymetry will define the domain in which we set up the ocean model. <br>

A dataset containing the bathymetric information can be in ***../../shared/bathymetry.jld2***


#### JLD2 filetype

Note that the file has the `*.jld2` extension. <br>
This is a Julia filetype which is exactly the same as `*.hdf5` in python https://www.geeksforgeeks.org/hdf5-files-in-python/ <br>
We can open this type of files with the `JLD2` package using the `jldopen` function

In [ ]:
using JLD2

file_path = "../../shared/bathymetry.jld2"
file_bathymetry = jldopen(file_path)

A _JLDFile_ is a `Dictionary`, a collection of datasets that can be accessed using _strings_ as keys <br>
(akin to how Dictionaries work in python). <br>
The bathymetry can be accessed with the "bathymetry" key.

In [ ]:
bathymetry = file_bathymetry["bathymetry"]

In [ ]:
typeof(bathymetry)

The bathymetric map is a `Matrix` (an alias for a two-dimensional array `Array`) <br>
of size 1440x600. You can think of these numbers as the _resolution_ (number of pixel) of the map. <br>

### Plotting 2D data

Let us plot the bathymetry and visualize underwater elevation <br>
Plotting in Julia can be done through the package `CairoMakie` <br>

The steps for plotting in CairoMakie are:
- define a figure as `fig = Figure()`. A resolution can be specified. It is possible to index the figure as a matrix
- specify an axis in the figure `ax = Axis(fig[1, 1])` (1, 1 stands for position 1, 1 in the figure layout)
- plot something inside the axis (using `ax` as an argument)

Since we will work with simulations of the ocean surface (two-dimensional), <br>
we can use the `heatmap!` function, which plots matrices (`contour!` could have been another choice). <br>
In a heatmap, the `colorrange` keyword specifies the contour limits. <br>
The colormap is chosen with the `colormap` keyword (choice of different colormaps, :balance, :hot, :viridis, :plasma, etc...)

In [ ]:
using CairoMakie

fig = Figure(resolution = (1000, 500))
ax  = Axis(fig[1, 1])

heatmap!(ax, bathymetry, colorrange = (-5500, 100), colormap = :terrain)

current_figure()

# Build a computational Grid

The fluid in the ocean is _continuous_, but a computer can only operate on _discrete_ quantities (vectors, arrays, matrices). <br>
To _represent_ a continuous fluid on a computer, we need to _discretize_ our geometry into "computational cells".
The entirety of all the computational cells and their arrangement form a ***grid*** (or a ***mesh***).

Grids are the backbone of ***all*** fluid flow simulations: constructing an adequate grid goes a long way in ensuring a good quality model.

If we want to discretize the ocean, we need a sphere! <br>
Oceananigans allows to construct a spherical grid with the `LatitudeLongitudeGrid` constructor.

Let us choose the number of points in the latitude and longitude direction (`Nlat` and `Nlon`) and construct the grid <br>
We will simulate the transport over the ocean surface, so we do not need a "vertical" direction. <br>
The size of the grid will, hence, be `(Nlat, Nlon, 1)`

In [ ]:
Nlon = 360
Nlat = 150

initial_grid = LatitudeLongitudeGrid(size = (Nlon, Nlat, 1), longitude = (-180, 180), latitude = (-75, 75), z = (0, 1))

`initial_grid` is a _structure_, i.e., it is a variable which contains 
Take a second to look at the pieces of information contained in this grid <br>

At the moment, this grid is just a spherical shell. <br>
To convert it into a realistic representation of the ocean, we have to load in the bathymetric map <br>

some convenience functions to interpolate our domain (the bathymetric map) onto the grid <br> are implemented in the file "useful_functions.jl" (within the folder "utils")

In Julia we can load source files `(*.jl)` through the function `include`. <br>
Including a source file will load all the functions defined in it.<br>
(you are welcome to look at the functions defined in "utils/useful_functions.jl")

In [ ]:
include("utils/useful_functions.jl")

grid = immersed_boundary_grid(initial_grid, bathymetry)

You can use the `visualize_cartesian_grid` function (defined in "useful_functions.jl") to plot the grid. <br> 
It uses the `CairoMakie` package to visualize a wireframe of the computational cells and a 
heatmap of the discrete continental mask.

In [ ]:
fig = visualize_cartesian_grid(grid)
current_figure()

Think about the cells in a grid as the "pixels" of an image; A larger number of pixels allow us to see more detailed images. <br>
The same happens in an ocean simulation; a larger number of cells (pixels) in a grid, <br>
allows us to "resolve" more detailed physics.

## Exercise: change the grid resolution

try changing the number of points in the longitudinal (`Nlat`) and the latitudinal (`Nlon`) direction, <br>
and see how the representation of the continents changes.

It is also possible to change the latitude and longitude of the domain to "zoom in" in particular regions <br>
For example, try `latitude = (-35, 30)` and `latitude = (-30, 30)` to zoom into west africa's coast

#### Note!
the bathymetry data has a size of (1440 x 600), so the maximum resolution allowable is `Nx = 1440`, `Ny = 600`

In [ ]:
## Write the new grid here
## look at the examples above:

# - define Nlon and Nlat
# - build an initial_grid (choose latitude = (ᵒwest, ᵒeast), longitude = (ᵒsouth, ᵒnorth))
# - build the grid with the immersed_boundary_grid function
# - visualize with the visualize_figure function
# - display the figure with current_figure()

# Load and interact with a velocity field

### Load velocity datasets

To load the (precomputed) velocity datasets, we again use the `JLD2` package.

In [ ]:
file_velocities = jldopen("../../shared/prescribed_mean_fields.jld2")

This file is a Dictionary containing two datasets:
- um
- vm 

Here `um` is the _zonal_ surface velocity (the surface velocity in the West-East direction) <br>
and `vm` is the _meridional_ surface velocity (the surface velocity in the South-North direction, along meridians). <br>
 
These datasets contain the velocities ***averaged*** over a period of 10 years. <br>
Let us assign these datasets to two variables and look at their structure.

In [ ]:
const Um = file_velocities["um"]
const Vm = file_velocities["vm"]

In [ ]:
typeof(Um)

In [ ]:
typeof(Vm)

Note that both the Arrays (`Um` and `Vm`) are ***3D*** arrays! Oceananigans handles always three-dimensional data for simplicity, <br>
a two-dimensional dataset (as in this case) is represented by a 3D array with ***one*** point in the unused direction

Let us plot the zonal velocity with `CairoMakie`. <br>
Since `Um` is a 3D array, we can plot it as `Um[:, :, 1]` (heatmap plots only 2D arrays)

In [ ]:
# Mean zonal velocity
u = Um[:, :, 1]

fig = Figure(resolution = (1000, 500))
ax  = Axis(fig[1, 1])
heatmap!(ax, u, colorrange = (-0.2, 0.2), colormap = :viridis)
current_figure()

Let us plot the meridional velocity (along latitude lines) in the same way

In [ ]:
# Mean meridional velocity
v = Vm[:, :, 1]

fig = Figure(resolution = (1000, 500))
ax  = Axis(fig[1, 1])
heatmap!(ax, v, colorrange = (-0.1, 0.1), colormap = :viridis)
current_figure()

### Oceananigans' `Field` type

Oceananigans represents variables discretized onto a particular grid with a `Field` type. <br> 
Fields are essentially containers that include the grid, the data allocated on the grid (the previously seen 3D arrays), <br>
and the associated boundary conditions.

We can define empty fields (i.e., empty containers associated with a grid) for our velocities and fill them with the velocity data we loaded.

The function `VelocityFields` returns three `Field`s (for U, V and W velocity) with data equal to 0 everywhere. <br> 
We can use this function to initiate the velocities and fill them with the data we loaded. <br>
(Note that the W velocity is zero as the simulation will be 2D)

In [ ]:
U, V, W = VelocityFields(grid)

set_velocity_from_array!(U, Um)
set_velocity_from_array!(V, Vm)

### Combining fields

Fields are handy because they can be combined through expressions. <br>
as an example, let us calculate the mean kinetic energy at the surface defined as

$$K = 0.5 * (U^2 + V^2)$$

To do that we can define an operation, through usual operators (`+`, `-`, `*`, `/`, `^` and so on...)
and wrap it in a `Field`. This will associate the field with the operation.

We then compute the operation with the `compute!` function

In [ ]:
operation_to_calculate = 0.5 * (U^2 + V^2)

K = Field(operation_to_calculate)

compute!(K)

We can plot kinetic energy with `CairoMakie` (as has been done before), but only after extracting the data! <br>

To access the data associated with an Oceananigans field we use the function `interior`

In [ ]:
k = interior(K)

# The data associated with a field is always three dimensional!
k

In [ ]:
fig = Figure(resolution = (1000, 500))
ax = Axis(fig[1, 1])
heatmap!(ax, k[:, :, 1], colorrange = (0, 0.05), colormap = :solar)

current_figure()

## Exercise: interact with the velocity field

Define and plot a new field as a function of `U` and `V`.

#### Remember!
- `heatmap!` plots 2D AbstractArrays.
- The `interior` function retrieves the data associated with a field.
- Fields (and associated data) are _always_ three-dimensional.

#### Tips
- Use the `typeof` and `size` function to confirm that the variable you want to plot is a 2D array (or matrix).
- Follow the example above for plotting and use `colorrange = (extrema(field))` and `colormap = :blues`.
- Do not forget to display the figure with `current_figure()` at the end of the cell!

In [ ]:
## Write the new field here
## look at the examples above:

# - define the operation you want to calculate (ex: U + 0.5 * V)
# - wrap it around a Field constructor (field = Field(expression))
# - compute the field with compute!(field)
# - extract the data with data = interior(field)
# - display the data with heatmap!